# 2. Model Calibration and Testing

In [8]:
%matplotlib inline
from model.binomial import EuropeanVanillaModel, HullWhiteEuropeanModel
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import date


#### GS Stock Prices (Nov 30, 2016 - Feb 27, 2017)

We chose these dates because they are between dividend paying dates of *Nov 29,2016* and *Feb 28, 2017*

We'll only need closing price from the stock data will compute the ratio of the closing price for each day

$$ ratio(p_i) = \frac{p_i}{p_{i+1}} $$

In [9]:
stock_prices = pd.read_csv('dataset/gs-stock-prices.csv', index_col=0, parse_dates=True).iloc[::-1]
stock_prices['Ratio'] = [price / stock_prices['Close'][i-1] for i, price in enumerate(stock_prices['Close'])]
stock_prices.drop([col for col in stock_prices.columns if col not in ['Close', 'Ratio']], axis=1, inplace=True)
stock_prices[:]

,Close,Ratio
Date,,
2016-11-30,219.289993,0.879517
2016-12-01,226.630005,1.033472
2016-12-02,223.360001,0.985571
2016-12-05,228.550003,1.023236
2016-12-06,231.380005,1.012382
2016-12-07,235.559998,1.018065
2016-12-08,241.449997,1.025004
2016-12-09,241.850006,1.001657
2016-12-12,237.169998,0.980649


#### Date: 03-17-2017 
#### USD LIBOR - 12 months: 1.81317 %
#### GS Option Expiration Date: 06-16-2017

In [10]:
K = 240
T = (date(2017,6,16) - date.today()).days
dt = 11
s0 = 242.14
r = .0181317 # LIBOR
n_steps = 8
option = 'call'
n_training = 40

In [11]:
put_strikes = pd.read_csv('dataset/gs-puts.csv')['Strike']
results = []
for k in put_strikes:
    hw = HullWhiteEuropeanModel(k, T, dt, r, n_steps, option, n_training, stock_prices, s0)
    print('stock tree for k:', k)
    print(pd.DataFrame(hw.binomial.s_tree.T))
    print('option tree for k:', k)
    print(pd.DataFrame(hw.binomial.o_tree.T))
    results.append(hw.price())
results

stock tree for k: 85
        0           1           2           3           4           5  \
0  242.14  258.351269  275.647882  294.102503  313.792660  334.801074   
1     NaN  220.663040  235.436427  251.198892  268.016653  285.960363   
2     NaN         NaN  201.091010  214.554050  228.918440  244.244526   
3     NaN         NaN         NaN  183.254950  195.523866  208.614186   
4     NaN         NaN         NaN         NaN  167.000886  178.181592   
5     NaN         NaN         NaN         NaN         NaN  152.188499   
6     NaN         NaN         NaN         NaN         NaN         NaN   
7     NaN         NaN         NaN         NaN         NaN         NaN   
8     NaN         NaN         NaN         NaN         NaN         NaN   

            6           7           8  
0  357.216000  381.131605  406.648359  
1  305.105405  325.532207  347.326582  
2  260.596693  278.043637  296.658653  
3  222.580902  237.482690  253.382152  
4  190.110847  202.838765  216.418817  
5  162.3

[224.90294104389926,
 223.88899639941678,
 222.87505175493726,
 221.86110711046098,
 220.84716246598501,
 219.83321782150958,
 218.81927317703148,
 218.69312307715421,
 218.59982000319215,
 218.50651692922469,
 217.96960104614968,
 203.05662237523379,
 188.14364370432517,
 173.23066503341815,
 158.31768636252349,
 187.61173951947939,
 277.06616226365503,
 366.52058500782852,
 455.97500775200427,
 545.42943049617929,
 622.08929381640746,
 291.5401338956575,
 -39.009026025097384,
 -369.55818594584969,
 -700.10734586659885,
 -1030.6565057873463,
 -1361.2056657080975,
 -935.13502668238391,
 -209.30183529191399,
 516.53135609855792,
 1242.3645474890261,
 1968.1977388794935,
 2694.0309302699616,
 3419.864121660431,
 2620.8384970641587,
 671.56269905730915,
 -1277.7130989495488,
 -3226.9888969563976]

In [12]:
call_strikes = pd.read_csv('dataset/gs-calls.csv')['Strike']
results = []
for k in call_strikes:
    hw = HullWhiteEuropeanModel(k, T, dt, r, n_steps, option, n_training, stock_prices, s0)
    results.append(hw.price())
results

[224.90294104389926,
 223.88899639941678,
 221.86110711046098,
 218.50651692922469,
 217.96960104614968,
 203.05662237523379,
 188.14364370432517,
 173.23066503341815,
 158.31768636252349,
 187.61173951947939,
 277.06616226365503,
 366.52058500782852,
 455.97500775200427,
 545.42943049617929,
 622.08929381640746,
 291.5401338956575,
 -39.009026025097384,
 -369.55818594584969,
 -700.10734586659885,
 -1030.6565057873463,
 -1361.2056657080975,
 -935.13502668238391,
 -209.30183529191399,
 516.53135609855792,
 1242.3645474890261,
 1968.1977388794935,
 2694.0309302699616,
 3419.864121660431,
 2620.8384970641587,
 671.56269905730915,
 -1277.7130989495488,
 -3226.9888969563976,
 -4032.994452380311,
 -2560.6848678830856,
 -1088.3752833858591,
 383.93430111136882,
 1856.2438856085944,
 2802.6763242908823,
 2307.9265457511237]